In [10]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.util import im2col

# 합성곱

가까이 있는 원소들의 관계를 모델에 포함시킬 수 있다.
필터의 각 원소에 대응되는 입력 데이터의 원소 곱의 전체 합이 출력된다.

## 합성곱 계층
1. 패딩
입력 데이터의 주변 값을 임의의 특정 값으로 채워 놓는다.
출력 크기를 조정하기 위해 사용한다.

2. 스트라이드
필터가 적용되는 간격을 의미한다.

출력크기 계산 하는 방법
$$ OH = \frac{H+2P-FH}{S} + 1 $$
$$ OW = \frac{W+2P-FW}{S} + 1 $$

# 3 차원 데이터 합성곱 연산
채널마다 입력데이터와 필터의 합성곱 연산을 수행하고, 결과를 모두 더해서 하나의 출력값을 만든다.

**입력 데이터의 채널 수 = 필터의 채널 수**

필터의 가중치 데이터는 4차원 데이터, (출력 채널 수, 입력 채널 수, 높이, 너비)

## 풀링 계층
최대 풀링, 평균 풀링

풀링에서 윈도우 크기와 스트라이드는 같은 값으로 설정하는 것이 일반적.

In [11]:
#img2col 사용
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [12]:
test = np.array([[[5, 7, 2], [3, 2, 8], [1, 6, 4]], [[15, 4, 3], [1, 7, 19], [5, 4, 2]]])
np.max(test, axis = 0)

array([[15,  7,  3],
       [ 3,  7, 19],
       [ 5,  6,  4]])

In [20]:
from common.layers import *
from common.util import *
from collections import OrderedDict

class SimpleConvNet:
  def __init__(
      self, 
      input_dim=(1, 28, 28), 
      conv_param={'filter_num': 30, 'filter_size':5, 'pad': 0, 'stride': 1},
      hidden_size=100, output_size=10, weight_init_std=0.01):
    filter_num = conv_param['filter_num']
    filter_size = conv_param['filter_size']
    filter_pad = conv_param['pad']
    filter_stride = conv_param['stride']
    input_size = input_dim[1]
    conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
    pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)
    self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
    self.params['b2'] = np.zeros(hidden_size)
    self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b3'] = np.zeros(output_size)

    self.layers = OrderedDict()
    self.layers['Conv1'] = Convolution(self.params['W1'], 
                                       self.params['b1'], 
                                       conv_param['stride'], 
                                       conv_param['pad'])
    self.layers['Relu1'] = Relu()
    self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
    self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
    
    self.last_layer = SoftmaxWithLoss()

  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
    return x

  def loss(self, x, t):
    y = self.predict(x)
    return self.last_layer.forward(y, t)
  
  def accuracy(self, x, t, batch_size=100):
    if t.ndim != 1 : t = np.argmax(t, axis=1)
    
    acc = 0.0
    
    for i in range(int(x.shape[0] / batch_size)):
        tx = x[i*batch_size:(i+1)*batch_size]
        tt = t[i*batch_size:(i+1)*batch_size]
        y = self.predict(tx)
        y = np.argmax(y, axis=1)
        acc += np.sum(y == tt) 
    
    return acc / x.shape[0]
  
  def gradient(self, x, t):
    self.loss(x, t)

    dout = 1
    dout = self.last_layer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)

    grads = {}
    grads['W1'] = self.layers['Conv1'].dW
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Affine1'].dW
    grads['b2'] = self.layers['Affine1'].db
    grads['W3'] = self.layers['Affine2'].dW
    grads['b3'] = self.layers['Affine2'].db

    return grads

In [19]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)

import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()


train loss:2.3000481139309024


AttributeError: 'SimpleConvNet' object has no attribute 'accuracy'